In [58]:

import numpy as np
from ortools.sat.python import cp_model as cp
import gurobipy as gp
from collections import deque
from itertools import chain

# Not! OKAY!

In [59]:
def readGrid(path):
    with open(f"../assets/data/Fobidoshi/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("12x12_1")
    for g in grid:
        print(g)

['x', '1', '.', '1', 'x', '.', '.', '.', 'x', '.', '1', '1']
['1', '.', '1', '.', '1', '1', '.', '.', '.', '.', '.', '.']
['.', '1', '1', '.', '1', '.', '.', '1', '.', '1', '.', '.']
['1', '.', '.', '.', '.', '.', '.', '.', '1', '.', '1', 'x']
['.', '.', '1', 'x', '1', '.', '.', '.', 'x', '.', '.', '.']
['x', '.', '1', '.', '.', '.', '.', '.', '.', '.', '1', '.']
['1', '.', '.', '.', '.', '.', '1', '1', '.', '.', '.', 'x']
['1', '.', '1', 'x', '.', '.', '.', '.', 'x', '.', '.', '.']
['x', '.', '.', '.', '.', '.', '.', '1', '.', '.', '.', '1']
['.', '.', '1', '.', '.', '1', '1', '1', '.', '.', '.', '1']
['1', '.', '1', '1', '.', '1', '1', '1', '.', '.', '.', '1']
['1', '.', '.', 'x', '1', '.', '1', 'x', '1', '.', '.', 'x']


In [67]:


def FobidoshiSolver(m, n, grid) :
    """_summary_

    Args:
        m (_type_): 行数
        n (_type_): 列数
        grid (_type_): 网格
    """
    
    Fobidoshi = gp.Model("Fobidoshi")
    Fobidoshi.modelSense = gp.GRB.MAXIMIZE
    Fobidoshi.Params.lazyConstraints = 1
    Fobidoshi.update()
    x = {}
    for i in range(m):
        for j in range(n):
            x[i, j] = Fobidoshi.addVar(
            vtype = gp.GRB.BINARY,
            obj = 1,
            name = f"x[{i},{j}]")
            if grid[i][j] == "x":
                Fobidoshi.addConstr(x[i, j] == 0)
            elif grid[i][j] == "1":
                Fobidoshi.addConstr(x[i, j] == 1)
    for i in range(m):
        for j in range(n - 3):
            Fobidoshi.addConstr(gp.quicksum([x[i, j], x[i, j + 1], x[i, j + 2], x[i, j + 3]]) <= 3, name = f"x[{i},{j},r]")
    for j in range(n):
        for i in range(m - 3):
            Fobidoshi.addConstr(gp.quicksum([x[i, j], x[i + 1, j], x[i + 2, j], x[i + 3, j]]) <= 3, name = f"x[{i},{j},c]")
    Fobidoshi.setObjective(gp.quicksum(x[i, j] for i in range(m) for j in range(n)), gp.GRB.MAXIMIZE)
    Fobidoshi.write("Fobidoshi.lp")
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for i in range(m)]
            for i in range(m):
                for j in range(n):
                    if x_sol[i][j] > 1e-6:
                        curr_grid[i][j] = 1
                        
            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white = sum(chain.from_iterable(grid))
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 1 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = []
                while len(visited) < total_white:
                    start = find_unvisited_white()
                    if not start:
                        return []

                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    cnt_white = 0
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 1 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 0:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))

                if cnt_white == sum(chain.from_iterable(grid)):
                    return []
                borders_cut.sort(key=lambda x: len(x))
                return borders_cut
    
    Fobidoshi._x = x
    Fobidoshi.optimize(border_elim)
    # Visualize 
    
    for i in range(m):
        for j in range(n):
            if grid[i][j] != ".":
                print(grid[i][j], end = " ")
            else:
                if x[i, j].x > 1e-6:
                    print(f"1", end = " ")
                else:
                    print(f"x", end = " ")
        print()
    print()

    
if __name__ == "__main__":
    m, n, grid = readGrid("6x6_1")
    FobidoshiSolver(m, n, grid)
    

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 36 columns and 156 nonzeros
Model fingerprint: 0x69561db5
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 25.0000000
Presolve removed 33 rows and 23 columns
Presolve time: 0.00s
Presolved: 15 rows, 13 columns, 34 nonzeros
Found heuristic solution: objective 26.0000000
Variable types: 0 continuous, 13 integer (13 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        26.00000   26.00000  0

In [56]:
from collections import deque

def find_surrounding_black_cells(grid):
    rows, cols = len(grid), len(grid[0])
    visited = set()
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
    total_white = sum(chain.from_iterable(grid))
    # Find the first unvisited white cell
    def find_unvisited_white():
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == 1 and (i, j) not in visited:
                    return (i, j)
        return None
    borders_cut = []
    while len(visited) < total_white:
        start = find_unvisited_white()
        if not start:
            return []

        # Use BFS to find all connected white cells and record the surrounding black cells
        queue = deque([start])
        surrounding_black_cells = set()  # Use a set to avoid duplicates
        cnt_white = 0
        while queue:
            x, y = queue.popleft()
            if (x, y) in visited:
                continue
            visited.add((x, y))
            cnt_white += 1
            # Check all four adjacent directions
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < rows and 0 <= ny < cols:
                    if grid[nx][ny] == 1 and (nx, ny) not in visited:
                        queue.append((nx, ny))
                    elif grid[nx][ny] == 0:
                        surrounding_black_cells.add((nx, ny))
        borders_cut.append(list(surrounding_black_cells))

    if cnt_white == sum(chain.from_iterable(grid)):
        return []
    borders_cut.sort(key=lambda x: len(x))
    return borders_cut

# Define the grid
# grid = [
#     ['x', '1', '1', '1', 'x', '1', '1', '1', 'x', '1', '1', '1'],
#     ['1', '1', '1', 'x', '1', '1', '1', 'x', '1', '1', '1', '1'],
#     ['1', '1', '1', 'x', '1', '1', 'x', '1', '1', '1', 'x', '1'],
#     ['1', 'x', 'x', '1', 'x', 'x', '1', '1', '1', 'x', '1', 'x'],
#     ['x', '1', '1', 'x', '1', '1', 'x', '1', 'x', '1', 'x', '1'],
#     ['x', 'x', '1', '1', 'x', '1', '1', 'x', '1', '1', '1', '1'],
#     ['1', '1', 'x', '1', '1', 'x', '1', '1', '1', 'x', '1', 'x'],
#     ['1', '1', '1', 'x', '1', '1', '1', 'x', 'x', '1', '1', 'x'],
#     ['x', '1', 'x', '1', '1', 'x', 'x', '1', '1', 'x', 'x', '1'],
#     ['1', 'x', '1', '1', 'x', '1', '1', '1', 'x', '1', '1', '1'],
#     ['1', 'x', '1', '1', 'x', '1', '1', '1', 'x', '1', '1', '1'],
#     ['1', '1', '1', 'x', '1', '1', '1', 'x', '1', '1', '1', 'x']
# ]

grid = [
    [0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
    [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
    [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
    [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1],
    [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]
]
# Call the function and print the results
result = find_surrounding_black_cells(grid)
flat = chain.from_iterable(grid)
print(sum(flat))
print(result)


4
[]


In [50]:
grid_ = [[0] * 12 for _ in range(12)]
ak = [(4, 0), (3, 4), (4, 3), (4, 9), (3, 7), (5, 4), (5, 1), (8, 0), (8, 9), (8, 6), (1, 6), (0, 8), (2, 5), (1, 3), (2, 8), (6, 2), (7, 1), (7, 7), (6, 5), (6, 11), (6, 8), (4, 5), (5, 6), (4, 8), (8, 2), (8, 5), (9, 4), (0, 4), (2, 10), (7, 3), (6, 10), (3, 2), (3, 11), (9, 0), (0, 0), (5, 8), (10, 4), (1, 1), (11, 3), (10, 1), (0, 9), (8, 10), (2, 3), (1, 10), (7, 11), (7, 8)]
for (x, y) in list(visited):
    grid_[x][y] = 1

for g in grid_:
    print(g)

[0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0]
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1]
[0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0]
[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [53]:
result.sort(key=lambda x: len(x))

for k in result:
    print(k)

[(2, 10), (0, 9), (1, 10), (3, 11)]
[(2, 3), (3, 2), (3, 4), (4, 3)]
[(4, 5), (5, 4), (3, 4), (4, 3)]
[(4, 5), (5, 4), (5, 6), (6, 5)]
[(8, 2), (6, 2), (7, 1), (7, 3)]
[(6, 2), (7, 1), (5, 1), (8, 0), (5, 0)]
[(11, 7), (10, 8), (8, 10), (8, 9), (9, 8), (7, 11), (11, 11)]
[(11, 7), (7, 7), (10, 8), (10, 4), (11, 3), (8, 9), (9, 8), (8, 6), (8, 5), (7, 8), (9, 4)]
[(4, 0), (3, 4), (4, 3), (4, 9), (3, 7), (5, 4), (5, 1), (8, 0), (8, 9), (8, 6), (1, 6), (0, 8), (2, 5), (1, 3), (2, 8), (6, 2), (7, 1), (7, 7), (6, 5), (6, 11), (6, 8), (4, 5), (5, 6), (4, 8), (8, 2), (8, 5), (9, 4), (0, 4), (2, 10), (7, 3), (6, 10), (3, 2), (3, 11), (9, 0), (0, 0), (5, 8), (10, 4), (1, 1), (11, 3), (10, 1), (0, 9), (8, 10), (2, 3), (1, 10), (7, 11), (7, 8)]


In [57]:
y = [(4, 0), (3, 4), (4, 3), (4, 9), (3, 7), (5, 4), (5, 1), (8, 0), (8, 9), (8, 6), (1, 6), (0, 8), (2, 5), (1, 3), (2, 8), (6, 2), (7, 1), (7, 7), (6, 5), (6, 11), (6, 8), (4, 5), (5, 6), (4, 8), (8, 2), (8, 5), (9, 4), (0, 4), (2, 10), (7, 3), (6, 10), (3, 2), (3, 11), (9, 0), (0, 0), (5, 8), (10, 4), (1, 1), (11, 3), (10, 1), (0, 9), (8, 10), (2, 3), (1, 10), (7, 11), (7, 8)]
print(len(y))

46
